In [1]:
import pandas as pd
import os
import json
import math

In [2]:
base_text = []
base_text = open('/data/home/svetlana.alekseeva/u.data', encoding='cp1251').readlines()

In [3]:
ratings = []
for i in range(len(base_text)):
    ratings.append(base_text[i].strip().split("\t"))

In [4]:
base_text = []
base_text = open('/data/home/svetlana.alekseeva/u.item', encoding='cp1251').readlines()

In [5]:
films = []
for i in range(len(base_text)):
    films.append(base_text[i].strip().split("|"))

In [6]:
words = ['user_id','film_id','mark','timestamp']
film_tags = ['movie_id','title','release_date','video_release_date','IMDb_URL','unknown','Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Film_Noir','Horror','Musical','Mystery','Romance','Sci_Fi','Thriller','War','Western']

In [7]:
pd_ratings = pd.DataFrame(ratings, columns=words)
pd_films  = pd.DataFrame(films, columns=film_tags)

In [8]:
pd_ratings.mark = pd_ratings.mark.astype(int)

In [9]:
pd_films['all_marks_count'] = 0.0
pd_films['average_rating'] = 0.0
pd_films['positive_marks'] = 0.0
pd_films['positive_avg'] = 0.0
pd_films['average_rating_mu'] = 0.0
pd_films['wilson_score'] = 0.0

In [10]:
mu = (pd_ratings.mark.sum())/(pd_ratings.mark.count())
koef_k = 10.0
koef_z = 2.576

In [11]:
mu

3.52986

In [42]:
pd_films.average_rating_mu = pd_films.average_rating_mu.astype(float)
pd_films.wilson_score = pd_films.wilson_score.astype(float)
pd_films.positive_avg = pd_films.positive_avg.astype(float)

In [53]:
for i in range(len(pd_films)):
    n = pd_ratings.mark[(pd_ratings.film_id == pd_films.movie_id[i])].count()
    marks = pd_ratings.mark[(pd_ratings.film_id == pd_films.movie_id[i])].sum()
    pd_films.all_marks_count[i] = n
    pd_films.average_rating[i] = marks/n
    pd_films.positive_marks[i] = pd_ratings.mark[(pd_ratings.film_id == pd_films.movie_id[i])&(pd_ratings.mark >= 4)].count()
    pd_films.positive_avg[i] = pd_films.positive_marks[i]/n
    pd_films.average_rating_mu[i] = (float(marks) + mu*float(koef_k))/(float(n)+float(koef_k))
    wilson_koef = koef_z*koef_z/float(n)
    pd_films.wilson_score[i] = (float(pd_films.positive_avg[i])+(wilson_koef/2) - koef_z*math.sqrt((float(pd_films.positive_avg[i])*(1-float(pd_films.positive_avg[i]))/n)+(wilson_koef/(4*n))))/(1 + wilson_koef)
#print(str(pd_films.movie_id[i])+' '+str(pd_films.average_rating_mu[i])+' '+str(pd_films.title[i])+' '+str(marks)+' '+str(mu)+' '+str(n))

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:7: Setti

In [54]:
top10_rates = [] 
top10_average = []
top10_rating = []
top10_lower = []

for index, row in (pd_films.sort_values(by = ['all_marks_count','title'], ascending = [False,True]).head(10)).iterrows():
    top10_rates.append(row.movie_id)

for index, row in (pd_films.sort_values(by = ['average_rating','title'], ascending = [False,True]).head(10)).iterrows():
    top10_average.append(row.movie_id)
    
for index, row in (pd_films.sort_values(by = ['average_rating_mu','title'], ascending = [False,True]).head(10)).iterrows():
    top10_rating.append(row.movie_id)
    
for index, row in (pd_films.sort_values(by = ['wilson_score','title'], ascending = [False,True]).head(10)).iterrows():
    top10_lower.append(row.movie_id)

In [55]:
with open('/data/home/svetlana.alekseeva/lab06s.json', 'w') as outfile:
    json.dump({ 'top10_rates': top10_rates,'top10_average': top10_average,'top10_rating': top10_rating,'top10_lower': top10_lower}, outfile)

In [56]:
top10_rating

['318', '483', '64', '408', '169', '12', '603', '50', '114', '178']

In [57]:
pd_films.sort_values(by = ['wilson_score','title'], ascending = [False,True])

,movie_id,title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,...,Sci_Fi,Thriller,War,Western,all_marks_count,average_rating,positive_marks,positive_avg,average_rating_mu,wilson_score
63,64,"Shawshank Redemption, The (1994)",01-Jan-1994,,http://us.imdb.com/M/title-exact?Shawshank%20R...,0,0,0,0,0,...,0,0,0,0,283,4,255,0.901060,4.413988,0.845753
317,318,Schindler's List (1993),01-Jan-1993,,http://us.imdb.com/M/title-exact?Schindler's%2...,0,0,0,0,0,...,0,0,1,0,298,4,265,0.889262,4.436034,0.833698
97,98,"Silence of the Lambs, The (1991)",01-Jan-1991,,http://us.imdb.com/M/title-exact?Silence%20of%...,0,0,0,0,0,...,0,1,0,0,390,4,344,0.882051,4.270747,0.833453
478,479,Vertigo (1958),01-Jan-1958,,http://us.imdb.com/M/title-exact?Vertigo%20(1958),0,0,0,0,0,...,0,1,0,0,179,4,162,0.905028,4.213220,0.833260
482,483,Casablanca (1942),01-Jan-1942,,http://us.imdb.com/M/title-exact?Casablanca%20...,0,0,0,0,0,...,0,0,1,0,243,4,216,0.888889,4.420153,0.826281
49,50,Star Wars (1977),01-Jan-1977,,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,0,0,...,1,0,1,0,583,4,501,0.859348,4.344517,0.818201
602,603,Rear Window (1954),01-Jan-1954,,http://us.imdb.com/M/title-exact?Rear%20Window...,0,0,0,0,0,...,0,1,0,0,209,4,185,0.885167,4.348395,0.816144
426,427,To Kill a Mockingbird (1962),01-Jan-1962,,http://us.imdb.com/M/title-exact?To%20Kill%20a...,0,0,0,0,0,...,0,0,0,0,219,4,193,0.881279,4.258946,0.813473
356,357,One Flew Over the Cuckoo's Nest (1975),01-Jan-1975,,http://us.imdb.com/M/title-exact?One%20Flew%20...,0,0,0,0,0,...,0,0,0,0,264,4,230,0.871212,4.263864,0.808876
11,12,"Usual Suspects, The (1995)",14-Aug-1995,,http://us.imdb.com/M/title-exact?Usual%20Suspe...,0,0,0,0,0,...,0,1,0,0,267,4,232,0.868914,4.354869,0.806655
